In [1]:
# Manually add the root path before running the test
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from utils.helpers import (
    comparator_less,
    prepare_ancilla_cell_validity,
    Indexer
)


In [13]:
from math import ceil, log2
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from utils.helpers import Indexer
from oracle.column_uniqueness import column_uniqueness_circuit
from oracle.row_uniqueness import row_uniqueness_circuit
from oracle.cell_validity import cell_validity_circuit

# --- 1) PARAMETERS & GRID ---
n = 3
# columns have duplicates  col_constraints_flag should be 1
grid = [
    3, 1, 2, 
    2, 0, 1, 
    1, 2, 0,  
]

# --- 2) SET UP INDEXER + QUBIT POOL ---
k = max(1, int(ceil(log2(n))))
num_anc = 2*n + k + 2 
idx = Indexer(n, num_anc)

qr = QuantumRegister(idx.total_qubits, name="q")
qc = QuantumCircuit(qr, name="column_uniqueness_demo")

# --- 3) LOAD GRID INTO DATA ---
for i in range(n):
    for j in range(n):
        val = grid[i*n + j]
        for bit in range(idx.k):
            if (val >> bit) & 1:
                qc.x(qr[idx.data(i, j, bit)])

# --- 4) APPLY THE ORACLE ---
row_uniqueness_circuit(qc, qr, idx)
column_uniqueness_circuit(qc, qr, idx)
cell_validity_circuit(qc, qr, idx, n)

# --- 5) COLLAPSE INTO GLOBAL FLAG ---
controls = [
    qr[idx.cell_valid_flag()],
    qr[idx.row_flag()],
    qr[idx.col_flag()]
]
global_q = qr[idx.global_flag()]
qc.mcx(controls, global_q, ctrl_state='0'*len(controls))
qc.x(global_q)

# --- 6) MEASURE 
qc.measure_all()

# --- 7) SIMULATE & PRINT RESULTS ---
sim = AerSimulator(method="matrix_product_state")
tcirc = transpile(qc, sim, optimization_level=3)
counts = sim.run(tcirc, shots=1).result().get_counts()

print("Total qubits:", qc.num_qubits)
print("Counts for col_constraints_flag:", counts)


Total qubits: 32
Counts for col_constraints_flag: {'00000000001100001001010010100111': 1}


In [15]:
# after you’ve run and obtained `result.get_counts()`:
bitstring = next(iter(counts))              # e.g. '01000101…'
# Qiskit’s bitstrings are returned MSB→LSB (left→right), and qubit 0 is the rightmost bit
bits = list(map(int, bitstring[::-1]))      # now bits[i] is the measured value of qubit i

# 1) print out each qubit’s role and its measured value
print("Measured qubits:")
for q in range(idx.total_qubits):
    role = idx.pretty(q)
    print(f" q[{q:2d}] = {bits[q]}   ⟶ {role}")

# 2) reconstruct the grid from the data-region
recovered = [None] * (n*n)
for i in range(n):
    for j in range(n):
        v = 0
        for b in range(idx.k):
            bitval = bits[idx.data(i, j, b)]
            v |= (bitval << b)
        recovered[i*n + j] = v

# 3) print the grid as rows
print("\nReconstructed grid values:")
for i in range(n):
    row = recovered[i*n:(i+1)*n]
    print(" ", row)

# 4) check the final flag
final_bit = bits[idx.global_flag()]
print(f"\nglobal_flag  (q[{idx.global_flag()}]) = {final_bit}")


Measured qubits:
 q[ 0] = 1   ⟶ data(0,0,0)
 q[ 1] = 1   ⟶ data(0,0,1)
 q[ 2] = 1   ⟶ data(0,1,0)
 q[ 3] = 0   ⟶ data(0,1,1)
 q[ 4] = 0   ⟶ data(0,2,0)
 q[ 5] = 1   ⟶ data(0,2,1)
 q[ 6] = 0   ⟶ data(1,0,0)
 q[ 7] = 1   ⟶ data(1,0,1)
 q[ 8] = 0   ⟶ data(1,1,0)
 q[ 9] = 0   ⟶ data(1,1,1)
 q[10] = 1   ⟶ data(1,2,0)
 q[11] = 0   ⟶ data(1,2,1)
 q[12] = 1   ⟶ data(2,0,0)
 q[13] = 0   ⟶ data(2,0,1)
 q[14] = 0   ⟶ data(2,1,0)
 q[15] = 1   ⟶ data(2,1,1)
 q[16] = 0   ⟶ data(2,2,0)
 q[17] = 0   ⟶ data(2,2,1)
 q[18] = 0   ⟶ row_flag
 q[19] = 0   ⟶ col_flag
 q[20] = 1   ⟶ cell_valid_flag
 q[21] = 1   ⟶ global_flag
 q[22] = 0   ⟶ ancilla(0)
 q[23] = 0   ⟶ ancilla(1)
 q[24] = 0   ⟶ ancilla(2)
 q[25] = 0   ⟶ ancilla(3)
 q[26] = 0   ⟶ ancilla(4)
 q[27] = 0   ⟶ ancilla(5)
 q[28] = 0   ⟶ ancilla(6)
 q[29] = 0   ⟶ ancilla(7)
 q[30] = 0   ⟶ ancilla(8)
 q[31] = 0   ⟶ ancilla(9)

Reconstructed grid values:
  [3, 1, 2]
  [2, 0, 1]
  [1, 2, 0]

global_flag  (q[21]) = 1
